In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import pickle

def recommend_trees(user_input, scaler_dataset_filename):

    with open(scaler_dataset_filename, 'rb') as f:
        data = pickle.load(f)
        scaler = data['scaler']
        df = data['dataset']
        numeric_features = data['numeric_features']
    
    filtered_df = df.copy() 
    

    if user_input.get('state'):
        filtered_df = filtered_df[filtered_df['State'] == user_input['state']]

    if user_input.get('tree_species'):
        filtered_df = filtered_df[filtered_df['Native Species'].str.contains(user_input['tree_species'], case=False)]
    

    weights = {
        'Growth Period (Months)': 3,
        'Land Area (hectare)': 1,
        'Green Credit': 5,
        'Region': 2,
        'Cost (per hectare)': 4
    }
    

    for feature in numeric_features:
        if user_input.get(feature) is None:
            user_input[feature] = 0
    

    X = filtered_df[numeric_features].values
    
    user_vector = scaler.transform([[user_input.get(feature, 0) for feature in numeric_features]])
    
    filtered_df['Number of Trees'] = filtered_df['Planting Density (trees per hectare)'] * filtered_df['Land Area (hectare)']
    filtered_df['Cost'] = filtered_df['Cost (per hectare)'] * filtered_df['Land Area (hectare)']
    filtered_df['Carbon Credit Earnings Estimate'] = filtered_df['Carbon Sequestration (kg CO2/year/tree)'] * filtered_df['Number of Trees']
    filtered_df['Green Credit Earnings Estimate'] = filtered_df[['Number of Trees', 'Planting Density (trees per hectare)']].min(axis=1)
    

    scaled_features = scaler.transform(X)
    user_vector_scaled = scaler.transform([user_vector.flatten()])   

    similarity_scores = cosine_similarity(scaled_features, user_vector_scaled)

    filtered_df['Similarity Score'] = similarity_scores.flatten()
    
    weighted_similarity_score = pd.Series(0, index=filtered_df.index)
    for feature, weight in weights.items():
        if feature in user_input:
            filtered_df['Weighted ' + feature] = filtered_df['Similarity Score'] * weight
            weighted_similarity_score += filtered_df['Weighted ' + feature]
    
    filtered_df['Weighted Similarity Score'] = weighted_similarity_score

    top_recommendations = filtered_df.sort_values(by='Weighted Similarity Score', ascending=False).head(10)
    
    return top_recommendations

user_input = {
    'state': 'Karnataka', 
    'tree_species': 'Neem',
    'Growth Period (Months)': 20,
}

recommendations = recommend_trees(user_input, 'EcoGrowAdvisor_Amogha.pkl')

print("Top 10 Recommendations:")
for index, row in recommendations.iterrows():
    print(f"Land Parcel ID: {row['Land Parcel ID']}")
    print(f"Tree Specie(s): {row['Native Species']}")
    print(f"Land Area (in hectare): {row['Land Area (hectare)']}")
    print(f"State: {row['State']}")
    print(f"District: {row['District']}")
    print(f"Division: {row['Division']}")
    print(f"Latitude and Longitude: {row['Latitude']}, {row['Longitude']}")
    print(f"Growth Period: {row['Growth Period (Months)']} months")
    print(f"Total Cost Estimate (in Rupees): {row['Cost']}")
    print(f"Green Credit Earnings Estimate: {row['Green Credit Earnings Estimate']}")
    print(f"Carbon Credit Earnings Estimate: {row['Carbon Credit Earnings Estimate']}")
    print(f"Maintenance Requirements: {row['Maintenance Requirements']}")
    print("\n")


Top 10 Recommendations:
Land Parcel ID: LP076
Tree Specie(s): Neem, Banyan
Land Area (in hectare): 22.896325661621024
State: Karnataka
District: Bangalore
Division: South
Latitude and Longitude: 29.4402721433824, 69.07970526442193
Growth Period: 24 months
Total Cost Estimate (in Rupees): 33699.24438821222
Green Credit Earnings Estimate: 527.0
Carbon Credit Earnings Estimate: 241327.27247348556
Maintenance Requirements: Low


Land Parcel ID: LP096
Tree Specie(s): Neem, Banyan
Land Area (in hectare): 27.110335430670744
State: Karnataka
District: Bangalore
Division: South
Latitude and Longitude: 14.143780244946193, 91.83899639517988
Growth Period: 24 months
Total Cost Estimate (in Rupees): 133682.3809298332
Green Credit Earnings Estimate: 524.0
Carbon Credit Earnings Estimate: 284116.3153134294
Maintenance Requirements: Low


Land Parcel ID: LP046
Tree Specie(s): Neem, Banyan
Land Area (in hectare): 26.357077325363186
State: Karnataka
District: Bangalore
Division: South
Latitude and Longi